In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# from AlgoTestFactory import *

# altf1 = AlgoTestFactory([0, 10000], 15, savePath="Combined_4_test2", storeAccuracies=True)

# altf1.run()

In [4]:
from SktWsegRWR_utf8 import *
import pickle
import ProbData
from ProbModels import *
import multiprocessing
import math
import json
import pprint
import csv

Dataloader Started[Prob]...
Dataloader Finished[Prob]...


In [5]:
pb = ProbModels(fullCo_oc_mat = ProbData.fullCo_oc_mat, unigram_counts = ProbData.unigram_counts,
               cng2cngFullMat = ProbData.cng2cngFullMat, cng2index_dict = ProbData.cng2index_dict,
               w2w_samecng_fullmat=ProbData.w2w_samecng_fullmat, samecng_unigram_counts=ProbData.samecng_unigram_counts,
               v2c_fullMat = ProbData.v2c_fullMat)

goodFileDict = pickle.load(open('mergedGood_v3.p', 'rb'))
algo = SktWsegRWR(
            w2w_modelFunc = pb.get_w2w_mat, 
            t2t_modelFunc = pb.get_cng2cng_mat,
            v2c_modelFunc = pb.get_v2c_ranking,
            sameCng_modelFunc = pb.get_w2w_samecng_mat,
            partition=[0.25, 0.25, 0.25, 0.1]
        )

In [6]:
def loadSentence(fName, folderTag):
        # print('File: ', fName)
        try:
            dcsObj = pickleFixLoad('../Text Segmentation/DCS_pick/' + fName)           
            if folderTag == "C1020" :
                sentenceObj = pickleFixLoad('../TextSegmentation/corrected_10to20/' + fName)
            else:
                sentenceObj = pickleFixLoad('../TextSegmentation/Pickle_Files/' + fName)

        except (KeyError, EOFError, pickle.UnpicklingError) as e:
            return None, None
        return(sentenceObj, dcsObj)
    
def Accuracy(prediction, dcsObj):
#     solution = [rom_slp(c) for c in dcsObj.dcs_chunks]
    solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#     ac = 100*sum(list(map(lambda x: x in solution, prediction)))/len(prediction)
    ac = 100*sum(list(map(lambda x: x in prediction, solution)))/len(solution)
    return ac

In [33]:
np.set_printoptions(precision=6)
d = pickle.load(open('.temp/midScore.p','rb'), encoding='utf8')
f = d[0]
print(f.keys())
print()
print(f['sentences'])
print(f['0'].keys())
print(f['initialQuery'])
print(f['0']['updated_query'])
print(f['0']['final_score'])
print(np.asarray(f['0']['final_score'].argsort()))
print()
print(f['6']['updated_query'])
print(f['6']['final_score'])

dict_keys(['5', 'TransitionMat_t2t', 'initialQuery', 'prediction', '3', 'TransitionMat_w2w_samecng', '4', 'DCSLemmas', 'accuracy', 'TransitionMat_w2w', '2', 'nodeList', '1', '0', 'sentence', '6'])

dict_keys(['updated_query', 'final_score', 'removed', 'final_rank', 'w2w_samecng_rank', 'w2w_samecng_score', 'w2w_rank', 't2t_score', 't2t_rank', 'w2w_score', 'winner'])
[15, 16]
[15, 16, 21]
[[ 0.040709  0.04046   0.041591  0.041591  0.037939  0.041064  0.041101
   0.041317  0.041317  0.041353  0.041227  0.041433  0.041429  0.041316
   0.040558  0.031458  0.03466   0.041182  0.041182  0.041182  0.040675
   0.037607]]
[[15 16 21  4  1 14 20  0  5  6 17 18 19 10 13  8  7  9 12 11  2  3]]

[15, 16, 21, 4, 1, 20, 14, 10, 2]
[[ 0.040852  0.038907  0.040852  0.040852  0.038907  0.040852  0.040852
   0.040852  0.040852  0.040852  0.038907  0.040852  0.040852  0.040852
   0.038907  0.038072  0.038907  0.040852  0.040852  0.040852  0.038907
   0.038907]]


In [12]:
np.set_printoptions(precision=4)
for f in list(goodFileDict.keys())[1:20]:
# for f in list(goodFileDict.keys())[1:100]:
# for f in fullfs[0:1]:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj, verbose = False)
        if(result != None):            
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
            ac = Accuracy(result, dcsObj)
#             print(f, "\t", ac)

In [13]:
with open('.temp/sandhi_encounters.csv', 'w') as fh:
    fcsv = csv.writer(fh)
    fcsv.writerow(['left', 'right', 'derivations', 'word_left', 'word_right', 'pos_left', 'pos_right'])
np.set_printoptions(precision=6)
# for f in list(goodFileDict.keys())[90:91]:
for f in list(goodFileDict.keys())[1:100]:
# for f in fullfs:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj)
        if(result != None):
            ac = Accuracy(result, dcsObj)
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#             print(result)
#             print('vs')
#             print(solution)
#             print(f, "\t", ac)

In [51]:
# fullfs = ['105587.p','293473.p','379245.p']
fullfs = ['302864.p', '292204.p', '161425.p', '128533.p', '265112.p', '16349.p', '320178.p', '334655.p', '331468.p', '365975.p', '434538.p', '148267.p', '340997.p', '204751.p', '289589.p', '423330.p', '276728.p', '105056.p', '325962.p', '383515.p', '60055.p', '228316.p', '192549.p', '169215.p', '393452.p', '433996.p', '440103.p', '68686.p', '157189.p', '440567.p']

In [52]:
def GetSomeJSON():
    lowScore = []
    midScore = []
    highScore = []
    np.set_printoptions(precision=4)
#     for i in range(1, 400):
#         f = list(goodFileDict.keys())[i]
    for f in fullfs:
        sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
        if(sentenceObj != None):
            result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)

            if(result != None):
                runDetails['file'] = f
                ac = runDetails['accuracy']

                # CODE FOR SELECTING FILES 
#                 if(i%100 == 0):
#                     print('Chekpoint:', i)
                if(ac <= 40):
                    lowScore.append(runDetails)
                elif (ac <= 60):
                    if(ac >= 50):
                        midScore.append(runDetails)
                elif (ac <= 100):
                    if(ac >= 80):
                        highScore.append(runDetails)

    print('Lowscore files found:', len(lowScore))
    print('Midscore files found:', len(midScore))
    print('Highscore files found:', len(highScore))

    lowScore = lowScore[0:10]
    midScore = midScore[0:10]
    highScore = highScore[0:10]
    pickle.dump(lowScore, open('lowScore.p', 'wb'))
    pickle.dump(midScore, open('midScore.p', 'wb'))
    pickle.dump(highScore, open('highScore.p', 'wb'))

In [53]:
GetSomeJSON()

Lowscore files found: 10
Midscore files found: 10
Highscore files found: 10


In [55]:
def SeeSomeJSON(region = 'mid', files = ['lowScore.p', 'midScore.p', 'highScore.p']):
    np.set_printoptions(precision=3)
    if region == 'low':
        p = pickleFixLoad(files[0])
    elif region == 'mid':
        p = pickleFixLoad(files[1])
    else:
        p = pickleFixLoad(files[2])
#     for pf in p:
#         print(pf['file'])
    n_steps = p[0]['steps']
    for s in range(n_steps):
        
        print('Step:', s)
        print('='*20)
        
        for key, val in p[0][str(s)].items():
            print(key, ":")
            print(val)
            print('-'*20)
        
SeeSomeJSON('low')
print('='*20)
SeeSomeJSON('mid')
print('='*20)
SeeSomeJSON('high')

Step: 0
winner :
(23, 'tam', 'tad', 69)
--------------------
w2w_score :
[[ 0.401  0.007  0.007  0.007  0.006  0.008  0.008  0.008  0.     0.03
   0.03   0.064  0.     0.     0.     0.     0.     0.001  0.     0.008
   0.008  0.     0.042  0.167  0.     0.002  0.003  0.009  0.014  0.021
   0.067  0.     0.     0.079]]
--------------------
final_rank :
[ 0 23 10 30  9 33 28  7 11 22 31 29  6  4  5 20  2 19 17 16 14 32 26 18 21
  3 27  1 12 24  8 25 13 15]
--------------------
w2w_samecng_score :
[[ 0.402  0.001  0.001  0.001  0.014  0.035  0.035  0.035  0.     0.08
   0.08   0.007  0.     0.     0.     0.     0.     0.     0.     0.001
   0.001  0.     0.003  0.239  0.     0.     0.     0.004  0.034  0.004
   0.011  0.     0.     0.012]]
--------------------
t2t_rank :
[ 0 31 30  4  7  2 17 14 16 10 20 29 28 19 22 32 18 21 26  9 23 12  6  3 24
  8  5  1 33 25 13 27 11 15]
--------------------
updated_query :
[0, 23]
--------------------
removed :
[(18, 'muktam', 'mukta', 71), (19, 'mukt

In [68]:
def ArePredictionsOK():
    with open('.temp/weird.csv', 'w') as fh:
        wcsv = csv.writer(fh)
        wcsv.writerow(['filename', 'sentence', 'solution', 'prediction'])
        np.set_printoptions(precision=4)
        badCount = 0
        for i in range(1, 600):
            f = list(goodFileDict.keys())[i]

            sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
            if(sentenceObj != None):
                result = algo.predict(sentenceObj, dcsObj, verbose = False)
                if(result != None):            
                    solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
                    ac = Accuracy(result, dcsObj)
                    if ac >= 100 and len(result) != len(solution):
                        badCount += 1
                        if(badCount%10 == 0):
                            print("="*40)
                        sline = [f, sentenceObj.sentence, solution, result]
                        wcsv.writerow(sline)
        print(badCount, 'weird predictions encountered.')
        
ArePredictionsOK()

35 weird predictions encountered.


In [7]:
SeeSentence(sentenceObj)

SKT ANALYZE
---------------
upacAreRa SIlena rUpayOvanasaMpadA   
Analyzing  upacAreRa
0 :  upacAreRa ['upacAra'] [{'noun': ['i. sg. m.']}]
Analyzing  SIlena
0 :  SIlena ['SIla'] [{'noun': ['i. sg. m.', 'i. sg. n.']}]
0 :  SIla ['SIla'] [{'compound': ['iic.']}]
0 :  SIle ['SIla'] [{'noun': ['loc. sg. m.', 'acc. du. n.', 'nom. du. n.', 'loc. sg. n.']}]
0 :  SIla ['SIl'] [{'verb': ['imp. [1] ac. sg. 2']}]
3 :  ina ['na'] [{'noun': ['voc. sg. m.', 'voc. sg. n.']}]
4 :  na ['na'] [{'indeclinable': ['part.']}]
Analyzing  rUpayOvanasampadA
0 :  rUpa ['rUpa'] [{'compound': ['iic.']}]
10 :  sampadA ['sampad_2'] [{'noun': ['i. sg. f.']}]
4 :  yOvana ['yOvana'] [{'compound': ['iic.']}]
4 :  yO ['yu_3'] [{'noun': ['loc. sg. m.', 'loc. sg. f.']}]
6 :  vana ['vana'] [{'compound': ['iic.']}]
6 :  vana ['van'] [{'verb': ['imp. [1] ac. sg. 2']}]



In [8]:
SeeDCS(dcsObj)

DCS ANALYZE
---------------
upacAreRa SIlena rUpayOvanasaMpadA   
[['upacāra'], ['śīla'], ['rūpa', 'yauvana', 'sampad']]
Lemmas: ['upacAra', 'SIla', 'rUpa', 'yOvana', 'sampad']
[['89'], ['91'], ['3', '3', '90']]



In [7]:
def CSVforSupervised():
    np.set_printoptions(precision=4)
    # Clean the csv files
    metPerfFH = open('.temp/metPerfFH.csv', 'w')
    metPerfFH_bin = open('.temp/metPerfFH_bin.csv', 'w')

    metPerfFH.write('w2w,t2t,w2w_samecng,InDCS\n')
    metPerfFH_bin.write('w2w,t2t,w2w_samecng,InDCS\n')

    metPerfFH.close()
    metPerfFH_bin.close()
    ######################
    gfl = list(goodFileDict.keys())
    for i in range(0, 11000):
        f = gfl[i]
        if(i%250 == 0):
            print('Checkpoint:', i)
        sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
        
        if(sentenceObj != None):
            result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)




In [ ]:
np.set_printoptions(precision=6)
jsonBatch = []
for f_ in lowScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
pickle.dump(jsonBatch, open('.temp/lowScore.p','wb'))
print('33 % Complete')
#################################################################
jsonBatch = []
for f_ in midScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
pickle.dump(jsonBatch, open('.temp/midScore.p','wb'))
print('67 % Complete')
#################################################################
jsonBatch = []
for f_ in highScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
print(pprint.pprint(jsonBatch))
pickle.dump(jsonBatch, open('.temp/highScore.p','wb'))
print('100 % Complete')

In [53]:
t1 = ('super', -23)
t2 = ('super', -23)
t1 == t2

True

In [16]:
s, d = loadSentence('434538.p', goodFileDict['434538.p'])

In [17]:
SeeSentence(s)

SKT ANALYZE
---------------
mamatvenABiBUtaH  sa tatrEva parivartate   
Analyzing  mamatvenABiBUtaḥ
0 :  mamatvena ['mamatva'] [{'noun': ['i. sg. n.']}]
0 :  mamatva ['mamatva'] [{'compound': ['iic.']}]
0 :  mamatve ['mamatva'] [{'noun': ['acc. du. n.', 'nom. du. n.', 'loc. sg. n.']}]
0 :  mama ['asmad'] [{'noun': ['g. sg. *']}]
0 :  mama ['mA_1', 'mA_4'] [{'verb': ['pft. ac. pl. 2']}, {'verb': ['pft. ac. pl. 2']}]
4 :  tvA ['yuzmad'] [{'noun': ['acc. sg. *']}]
4 :  tu ['tu'] [{'indeclinable': ['conj.']}]
6 :  ina ['na'] [{'compound': ['iic.']}]
6 :  inA ['na'] [{'noun': ['nom. sg. f.']}]
6 :  enA ['enA_2'] [{'undetermined': ['adv.']}]
7 :  nABi ['nAB'] [{'compound': ['iic.']}]
7 :  na ['na'] [{'indeclinable': ['part.']}]
8 :  aBi ['aB'] []
8 :  aBi ['aB'] [{'indeclinable': ['prep.']}]
8 :  aBi ['aB'] [{'compound': ['iic.']}]
11 :  BUtas ['BUta', 'BU_1'] [{'noun': ['nom. sg. m.']}, {'verb': ['pp.']}]
11 :  BU ['BU_2'] [{'compound': ['iic.']}]
11 :  Bu ['Bu', 'BU_2'] [{'indeclinable': [